============ Datathon INSTRUCTIONS and INFORMATION ============

Challenge: SkyHigh Books want to know in 2018 what readers will rate their books that they publish. You are given data from 2017, where you know what readers read and what rating each user provided on average to all books they read.

Your job is 4 fold:

Predict what readers will rate on average in 2018 on all books they will read in 2017 + 2018. Use the wishlist of books readers want to read in 2018 and what they have already read in 2017 as a starting point. SkyHigh Books wants to know this information, as they want to market directly to readers who either have higher potential average scores on all books, or lower scores.

SkyHigh Books wants to know why some books in 2017 or 2018 (predict which books) have good or bad overall ratings. Is it certain words in the book? The genre of the book? The price in 2017 / 2018? Where it was sold? Why do some books on average have a higher rating than other books?

SkyHigh Books wants to maintain a good online presence on book review sites (hence we want to predict user averages), and so they want you to tell them how they can lift overall global reader satisifcation (not just each individual user, but overall).

Provide confidence bounds on your predictions. How much % are you sure your predictions are true or correct? Is the results plausible? Does the data seem plausible? (Ie do the word count follow some Power Law Distribution?)

**Datasets**

You are provided with 6 datasets:

1. **Books Information**
2. **Genres Mapping**
3. **User Data**
4. **Words in Book**
5. **Words Mapping**
6. **Example Submissions**

------

1. **Books Information** Actual info on books. Book ID, Barcode, difficulty (average *perceieved* reading difficulty of book --> 1 = easy, 5 = hard).
2. **Genres Mapping** Maps Genre IDs to real Genre types (eg Science)
3. **User Data** User's data on average ratings and what they read in 2017 + wishlist for 2018.
4. **Words in Book** What are the words in the actual book.
5. **Words Mapping** Actual words mapped to word ID
6. **Example Submissions** Shows an example of what you need to provide.

In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn import datasets, metrics
from surprise.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from surprise.model_selection import train_test_split, cross_validate
from xgboost import XGBRegressor
from xgboost import plot_importance
from matplotlib import pyplot


In [2]:
## loading data
Books_Information = pd.read_csv('Books information.csv')
User_Data = pd.read_csv('User Data.csv').replace("Not specified", np.nan)



In [ ]:
Books_Information.isnull().sum()

In [3]:
user_2017 = User_Data.iloc[:, [0, 1, 2, 4]]
arr_2017 = [[columns[0], columns[1], int(book), float(columns[3])] for columns in user_2017.values for book in columns[2].split(', ')]
user_2017 = pd.DataFrame(arr_2017, columns=user_2017.columns)

user_2018 = User_Data.iloc[:, [0, 1, 3]]
arr_2018 = [[columns[0], columns[1], int(book)] for columns in user_2018.values for book in columns[2].split(', ')]
user_2018 = pd.DataFrame(arr_2018, columns=user_2018.columns)

In [4]:
df_2017 = user_2017.merge(Books_Information, left_on='User Read Books (2017)', right_on='Book ID')
df_2018 = user_2018.merge(Books_Information, left_on='User Read Books (2018)', right_on='Book ID')

df_2017 = df_2017[["User ID", "User Read Books (2017)", "User Difficulty Choice", "Book Genre", "Difficulty (Reader suggested)", "Number Of Words", "Price (2017)", "Most Sold At", "Number Sold", "Average Rating (2017)"]].rename(columns={'User Read Books (2017)': 'Book', 'Price (2017)': 'Price', 'Average Rating (2017)': "Rating"})
df_2018 = df_2018[["User ID", "User Read Books (2018)", "User Difficulty Choice", "Book Genre", "Difficulty (Reader suggested)", "Number Of Words", "Price (2018)", "Most Sold At", "Number Sold"]].rename(columns={'User Read Books (2018)': 'Book', 'Price (2018)': 'Price'})

In [5]:
X = df_2017.iloc[:, 2:8].values
y = df_2017.iloc[:, -1].values

In [6]:
# Taking care of missing data
from sklearn.preprocessing import Imputer

imputer = Imputer(missing_values = 'NaN', strategy = 'mean', axis = 0)
imputer = imputer.fit(X[:, 0].reshape(-1, 1))
X[:, 0] = imputer.transform(X[:, 0].reshape(-1, 1)).round().reshape(-1, )

In [7]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

labelencoder = LabelEncoder()
X[:, 5] = labelencoder.fit_transform(X[:, 5])
onehotencoder = OneHotEncoder(categorical_features = [1, 5])
X = onehotencoder.fit_transform(X).toarray()

In [9]:
# feature scaling

from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X = sc_X.fit_transform(X)
sc_y = StandardScaler()
y = sc_y.fit_transform(y.reshape(-1, 1))

array([[-0.18977373, -0.18131846, -0.18040845, ..., -0.70603171,
         1.35861619,  0.6655698 ],
       [-0.18977373, -0.18131846, -0.18040845, ..., -0.70603171,
         1.35861619,  0.6655698 ],
       [-0.18977373, -0.18131846, -0.18040845, ..., -0.70603171,
         1.35861619,  0.6655698 ],
       ..., 
       [-0.18977373, -0.18131846, -0.18040845, ..., -1.41941219,
        -0.00531204, -0.95661034],
       [-0.18977373, -0.18131846, -0.18040845, ..., -1.41941219,
        -0.00531204, -0.95661034],
       [-0.18977373, -0.18131846, -0.18040845, ...,  0.00734876,
         0.89903934,  0.99398602]])

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)


In [ ]:
algo.fit(trainset)